ChatGPT was asked:

Can you give me python code that will perform LSTM to predict google stock price? I want to load the data from yfinance and perform looped hyperparameter optimization on 3 folds of historical data. I'd also like a summary graph that shows the fits for each of the optimization runs. and a prediction for 30 days from the best algorithm. Please break the output into 3 parts

Please give my python code that will: import google stock price from yfinance, predict future values using lstm, optimize hyperparmeters with 3 fold cross validation, plot fits for each model. Please break the code into 4 outputs

In [1]:
# Data Pre Processing and Model definition

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

# Set the time step
timestep = 60

# Download the data from Yahoo Finance
stock_data = yf.download('GOOG', start='2010-01-01', end='2022-02-21')

# Create a new dataframe with only the 'Close' column
data = stock_data.filter(['Close'])

# Convert the dataframe to a numpy array
dataset = data.values

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Create the training dataset
train_data = scaled_data[:int(0.8*len(dataset))]

# Split the data into x_train and y_train datasets
x_train = []
y_train = []
for i in range(timestep, len(train_data)):
    x_train.append(train_data[i-timestep:i, 0])
    y_train.append(train_data[i, 0])

# Convert the x_train and y_train datasets to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Define the LSTM model
def create_model():
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dense(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


[*********************100%***********************]  1 of 1 completed


In [6]:
# Hyperparameter tuning with grid search and time series cross-validation

# Wrap the Keras model in a scikit-learn regressor
model = KerasRegressor(build_fn=create_model, epochs=50, batch_size=64, verbose=0)

# Define the hyperparameters to optimize
params = {
    #'layers': [[100, 100], [150, 100], [100, 50, 25]],
    #'batch_size': [64, 128],
    #'epochs': [50, 100],
    #'dropout_rate': [0.2, 0.3],
    #'optimizer': ['adam', 'rmsprop']
}

# Perform a grid search with time series cross-validation
tscv = TimeSeriesSplit(n_splits=3)
grid_search = GridSearchCV(estimator=model, param_grid=params, cv=tscv, n_jobs=-1)
grid_search.fit(x_train, y_train)

# Print the best parameters and score
print('Best parameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_.model

# Train the best model on all of the training data
best_model.fit(x_train, y_train, epochs=50, batch_size=64, verbose=0)


C:\Users\User\AppData\Local\Temp\ipykernel_14044\4215781124.py:4: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, epochs=50, batch_size=64, verbose=0)


Best parameters: {}
Best score: -5.0032270034231864e-05


In [7]:
# Model evaluation and training history plot

# Evaluate the model on the training data
train_predict = best_model.predict(x_train)
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform([y_train])

# Calculate the root mean squared error
rmse = np.sqrt(np.mean(np.square(train_predict - y_train)))
print('Train RMSE:', rmse)

# Plot the training history
plt.figure(figsize=(10, 6))
plt.plot(best_model.history.history['loss'], label='Training Loss')
plt.title('Training History')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()


75/75 [==============================] - 2s 21ms/step
Train RMSE: 21.93460536956099


KeyError: 'loss'

<Figure size 1000x600 with 0 Axes>

In [ ]:
# Forecasting the best fit from the past

# Create the testing dataset
test_data = scaled_data[int(0.8*len(dataset))-timestep:]
x_test = []
for i in range(timestep, len(test_data)):
    x_test.append(test_data[i-timestep:i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Generate predictions for the testing dataset
test_predict = best_model.predict(x_test)
test_predict = scaler.inverse_transform(test_predict)

# Plot the actual stock prices and the predicted prices
plt.figure(figsize=(10, 6))
plt.plot(data.index[int(0.8*len(dataset)):], data['Close'][int(0.8*len(dataset)):], label='Actual')
plt.plot(data.index[int(0.8*len(dataset))+timestep:len(data)], test_predict, label='Predicted')
plt.title('Google Stock Price')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()


In [ ]:
# Forecasting the next 60 days

# Make predictions for the next 60 days
future_predictions = []
last_sequence = x_test[-1:]
for i in range(60):
    next_prediction = best_model.predict(last_sequence)
    future_predictions.append(next_prediction)
    last_sequence = np.append(last_sequence[:,1:,:], [[next_prediction]], axis=1)
future_predictions = scaler.inverse_transform(future_predictions)

# Create a dataframe with the predicted prices for the next 60 days
prediction_dates = pd.date_range(start=data.index[-1]+pd.Timedelta(days=1), periods=60, freq='D')
future_prices = pd.DataFrame(future_predictions, index=prediction_dates, columns=['Predicted'])

# Plot the actual stock prices and the predicted prices for the next 60 days
plt.figure(figsize=(10, 6))
plt.plot(data.index[int(0.8*len(dataset)):], data['Close'][int(0.8*len(dataset)):], label='Actual')
plt.plot(data.index[int(0.8*len(dataset))+timestep:len(data)], test_predict, label='Past Predictions')
plt.plot(future_prices.index, future_prices['Predicted'], label='Future Predictions')
plt.title('Google Stock Price')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()
